In [104]:
# import libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
import os
from tqdm import tqdm

## Read sample data

In [105]:
# read sample data
# block
# df_block = pd.read_csv('.\\data\\sample\\blocks.csv', header=None)
# df_block.columns = ['block_id','block_hash','merkel_route','prev_block_hash','block_minted_time','unknown1','unknown2']

# transaction
#df_transaction = pd.read_csv('.\\data\\sample\\transactions.csv', header=None)
#df_transaction.columns = ['transaction_hash','unknown1','unknown2','unknown3','block_hash','coinbase_flag','unknonw4']

# input
#df_input = pd.read_csv('.\\data\\sample\\input.csv', header=None)
#df_input.columns = ['input id','input_seq','transaction_hash','signature','unknown1','input_address'] # missing a couple of things in the sample (amount)

# output
#df_output = pd.read_csv('.\\data\\sample\\output.csv', header=None)
#df_output.columns = ['transaction id','output_seq','amount_satoshis','address','transaction_hash']

# entities
#df_entities = pd.read_json('.\\data\\sample\\entities.json')


## Full data
### Read data

In [106]:
# set path
path = '.\\gitrepos\\bse_clovrlabs_btc_fraud\\data\\spiderbum-blockchain-upf\\'

#read block files

'''
block_folders = [(path+x, x) for x in os.listdir(path)]

block_columns = ['block_id','block_hash','merkel_route','prev_block_hash','block_minted_time','unknown1','unknown2']
transaction_columns = ['transaction_hash','unknown1','unknown2','unknown3','block_hash','coinbase_flag','unknonw4']
input_columns = ['input id','input_seq','transaction_hash','signature','unknown1','input_address','amount_satoshis']
output_columns = ['transaction_id','output_seq','amount_satoshis','address','transaction_hash']

df_block = pd.DataFrame(columns=block_columns)
df_transaction = pd.DataFrame(columns=transaction_columns)
df_input = pd.DataFrame(columns=input_columns)
df_output = pd.DataFrame(columns=output_columns)

df_block = pd.concat([pd.read_csv(f[0]+'\\blocks.csv',header=None, names=block_columns) for f in block_folders])
df_transaction = pd.concat([pd.read_csv(f[0]+'\\transactions.csv', header=None, names=transaction_columns) for f in block_folders])
df_input = pd.concat([pd.read_csv(f[0]+'\\input.csv', header=None, names=input_columns) for f in block_folders])
df_output = pd.concat([pd.read_csv(f[0]+'\\output.csv', header=None, names=output_columns) for f in block_folders])

df_block.to_pickle('full_block.pkl')
df_transaction.to_pickle('full_transaction.pkl')
df_input.to_pickle('full_input.pkl')
df_output.to_pickle('full_output.pkl')
'''


"\nblock_folders = [(path+x, x) for x in os.listdir(path)]\n\nblock_columns = ['block_id','block_hash','merkel_route','prev_block_hash','block_minted_time','unknown1','unknown2']\ntransaction_columns = ['transaction_hash','unknown1','unknown2','unknown3','block_hash','coinbase_flag','unknonw4']\ninput_columns = ['input id','input_seq','transaction_hash','signature','unknown1','input_address','amount_satoshis']\noutput_columns = ['transaction_id','output_seq','amount_satoshis','address','transaction_hash']\n\ndf_block = pd.DataFrame(columns=block_columns)\ndf_transaction = pd.DataFrame(columns=transaction_columns)\ndf_input = pd.DataFrame(columns=input_columns)\ndf_output = pd.DataFrame(columns=output_columns)\n\ndf_block = pd.concat([pd.read_csv(f[0]+'\\blocks.csv',header=None, names=block_columns) for f in block_folders])\ndf_transaction = pd.concat([pd.read_csv(f[0]+'\\transactions.csv', header=None, names=transaction_columns) for f in block_folders])\ndf_input = pd.concat([pd.read_c

In [107]:
# Read block, txn, input and output source pkl files
df_block = pd.read_pickle('.\\data\\full_block.pkl')
df_transaction = pd.read_pickle('.\\data\\full_transaction.pkl')
df_input = pd.read_pickle('.\\data\\full_input.pkl')
df_output = pd.read_pickle('.\\data\\full_output.pkl')

# Read etities
df_entities = pd.read_json('.\\data\\entities.json')
#df_entities.to_csv('entities.csv')

In [108]:
# add block data to txn data
df_transaction = df_transaction.merge(df_block[['block_id','block_hash','block_minted_time']], on=['block_hash'], how='left')

# add input and output data to txn data
merged = df_transaction[:10].merge(df_input, left_on='transaction_hash', right_on='transaction_hash',how='left', suffixes=('','_in'))
merged = merged.merge(df_output, left_on='transaction_hash', right_on='transaction_hash',how='left', suffixes=('','_out'))
merged.rename(columns = {'address':'output_address','amount_satoshis':'amount_satoshis_in'}, inplace=True)

## Test pannel re-creation

In [109]:
# set transaction hash as example
txn = '8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc42d51ac03ed25f18e9'

# get selected input and calculate shares and amounts
inputs = merged[merged.transaction_hash == txn][['transaction_hash','coinbase_flag','input_seq','input_address','amount_satoshis_in','block_id','block_minted_time']].drop_duplicates()
inputs['total_amount_in'] = inputs['amount_satoshis_in'].sum()
inputs['amount_share_in'] = inputs['amount_satoshis_in'] / inputs['total_amount_in']

# get selection output and calculate shares and amounts
outputs = merged[merged.transaction_hash == txn][['transaction_hash','output_seq','output_address','amount_satoshis_out']].drop_duplicates()
outputs['total_amount_out'] = outputs['amount_satoshis_out'].sum()
outputs['amount_share_out'] = outputs['amount_satoshis_out'] / outputs['total_amount_out']

# merge input and outputs sample into one
full = outputs.merge(inputs, on=['transaction_hash'], how='left')

# calculate ammounts and fees
full['amount_spread_out'] = full['amount_share_in']*full['amount_satoshis_out']
full['fee'] = full['amount_share_in']*(full['total_amount_in']-full['total_amount_out'])
full['fee_spread'] = full['amount_share_in']*(full['total_amount_in']-full['total_amount_out'])*full['amount_share_out']

full[['transaction_hash','block_id','input_address','output_address','amount_spread_out','fee','fee_spread']]

,transaction_hash,block_id,input_address,output_address,amount_spread_out,fee,fee_spread
0,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc42d51ac03ed25f18e9,453318,166zajP74bcRVo7BmdeDME3mRX3Mi9e3xn,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,2503.64769,1386.37676,3.31484
1,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc42d51ac03ed25f18e9,453318,1LU3DtRE3XK32WxFqrnaT9k99nRgwHtLHd,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,635940.18056,352147.26556,841.98860
2,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc42d51ac03ed25f18e9,453318,1HVQNFf7vDpJVZk7tEzbFxnmALSezA2qPD,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,590236.94681,326839.43114,781.47725
3,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc42d51ac03ed25f18e9,453318,1LU3DtRE3XK32WxFqrnaT9k99nRgwHtLHd,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,101303.22495,56095.92655,134.12608
4,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc42d51ac03ed25f18e9,453318,166zajP74bcRVo7BmdeDME3mRX3Mi9e3xn,1FaWBBGDL4WxTQZJEcPJbnfRWSWZRshyyd,1044603.97555,1386.37676,1383.06192
5,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc42d51ac03ed25f18e9,453318,1LU3DtRE3XK32WxFqrnaT9k99nRgwHtLHd,1FaWBBGDL4WxTQZJEcPJbnfRWSWZRshyyd,265335112.55389,352147.26556,351305.27695
6,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc42d51ac03ed25f18e9,453318,1HVQNFf7vDpJVZk7tEzbFxnmALSezA2qPD,1FaWBBGDL4WxTQZJEcPJbnfRWSWZRshyyd,246266223.62205,326839.43114,326057.95389
7,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc42d51ac03ed25f18e9,453318,1LU3DtRE3XK32WxFqrnaT9k99nRgwHtLHd,1FaWBBGDL4WxTQZJEcPJbnfRWSWZRshyyd,42267029.84851,56095.92655,55961.80047


## Read pannel data

In [110]:
# set table format
pannel_columns = ['txn_hash', 'input_address', 'output_address', 'amount', 'fees', 'block_index', 'block_time']
df_pannel = pd.DataFrame(columns=pannel_columns)

# set path
pannel_path = '.\\gitrepos\\bse_clovrlabs_btc_fraud\\data\\pannel\\'

# read all files, put them together and export to pickle
#pannel_files = [(pannel_path+x, x) for x in os.listdir(pannel_path)]
#df_pannel = pd.concat([pd.read_csv(f[0]) for f in pannel_files])
#df_pannel.to_pickle('full_pannel.pkl')

# read pannel pickle file
df_pannel = pd.read_pickle('.\\data\\full_pannel.pkl')    

In [111]:
# example to compare versus re-created pannel
pannel = df_pannel[df_pannel.txn_hash == txn]
pannel

,txn_hash,input_address,output_address,ammount,fees,block_index,block_time
1,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc42d51ac03ed25f18e9,166zajP74bcRVo7BmdeDME3mRX3Mi9e3xn,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,2503.64769,1386.37676,453318,2017-02-16 12:05:04 +0000 UTC
2,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc42d51ac03ed25f18e9,1LU3DtRE3XK32WxFqrnaT9k99nRgwHtLHd,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,635940.18056,352147.26556,453318,2017-02-16 12:05:04 +0000 UTC
3,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc42d51ac03ed25f18e9,1HVQNFf7vDpJVZk7tEzbFxnmALSezA2qPD,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,590236.94681,326839.43114,453318,2017-02-16 12:05:04 +0000 UTC
4,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc42d51ac03ed25f18e9,1LU3DtRE3XK32WxFqrnaT9k99nRgwHtLHd,1ASaHGPN8qRuqZkpnR7d2tcndU9uHL6aGj,101303.22495,56095.92655,453318,2017-02-16 12:05:04 +0000 UTC
5,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc42d51ac03ed25f18e9,166zajP74bcRVo7BmdeDME3mRX3Mi9e3xn,1FaWBBGDL4WxTQZJEcPJbnfRWSWZRshyyd,1044603.97555,1386.37676,453318,2017-02-16 12:05:04 +0000 UTC
6,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc42d51ac03ed25f18e9,1LU3DtRE3XK32WxFqrnaT9k99nRgwHtLHd,1FaWBBGDL4WxTQZJEcPJbnfRWSWZRshyyd,265335112.55389,352147.26556,453318,2017-02-16 12:05:04 +0000 UTC
7,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc42d51ac03ed25f18e9,1HVQNFf7vDpJVZk7tEzbFxnmALSezA2qPD,1FaWBBGDL4WxTQZJEcPJbnfRWSWZRshyyd,246266223.62205,326839.43114,453318,2017-02-16 12:05:04 +0000 UTC
8,8c852e187a0541cd8ea8c93a6c728843b5f8b9c579b6fc42d51ac03ed25f18e9,1LU3DtRE3XK32WxFqrnaT9k99nRgwHtLHd,1FaWBBGDL4WxTQZJEcPJbnfRWSWZRshyyd,42267029.84851,56095.92655,453318,2017-02-16 12:05:04 +0000 UTC


## Elliptic Data
### Read data

In [172]:
# read data
ell_transaction = pd.read_csv('.\\data\\elliptic_dataset\\transactions.csv')
ell_input = pd.read_csv('.\\data\\elliptic_dataset\\inputs.csv')
ell_output = pd.read_csv('.\\data\\elliptic_dataset\\outputs.csv')
ell_deanon = pd.read_csv('.\\data\\elliptic_dataset\\deanonymised_elliptic.csv')

In [173]:
# convert from str to list and create a list of unique input/output addresses
import ast
ell_input['addresses'] = [ast.literal_eval(n) for n in ell_input['addresses']]
ell_addresses_in = pd.DataFrame(ell_input['addresses']).explode('addresses', ignore_index = True)

ell_output['addresses'] = [ast.literal_eval(n) for n in ell_output['addresses']]
ell_addresses_out = pd.DataFrame(ell_output['addresses']).explode('addresses', ignore_index = True)

ell_all_addresses = ell_addresses_in.append(ell_addresses_out, ignore_index=True)
ell_all_addresses = ell_all_addresses.drop_duplicates().reset_index()
ell_all_addresses.drop(columns=['index'], inplace=True)
ell_all_addresses

,addresses
0,189hYTwr2ufByQiemfoA8Go6h6MYwGeHas
1,1D7wdkmi2PQTvPEQxqhtHMM3NoPTDK1dVd
2,19WuSzgcWrx9u4bNWwiak3x1yK2wWQDvVQ
3,1Nf7uW9i6Ao1aRuPH6N2wFUjWo16fZxehp
4,1ELKdi1Q4TiehbxxiS1peuZJncUHhbGtso
...,...
823439,1MdVTD9YRfbArdFXMEcCpvmwsed7zHEEZo
823440,17g1GRRmrswz4ZnQp8XsPmMXwZcqqYzkga
823441,16491Nb3qRm39H2Q66SLVFdgt7zpik9CR8
823442,1GwTZ4wh9gZJvyqwK3HCa3C3CQUCTZUCxN


In [175]:
# find addresses that are labelled
labels_data = pd.read_csv('.\\data\labels_extended_original.csv', 
                          chunksize = 1000000,
                          names=['address','id', 'entity','category','user_score'])

for chunk in labels_data:
    chunk = chunk[chunk['address'].isin(ell_all_addresses['addresses'])]
    chunk.to_csv('.\\data\\labels_ell_address.csv', mode='a', index=False, header=None)

## Generate other files
### Labels processing

In [112]:
# read original labels data
labels_extended = pd.read_csv('.\\data\\labels_extended_original.csv', header=None, names=['address','index','name','category','user_score'])

# eliminate duplicate rows
labels_extended = labels_extended.drop_duplicates()

# export to csv
labels_extended[['address','index','name','category','user_score']].to_csv('c:\\temp\\labels_extended.csv')

### Unique input/output addresses

In [113]:
# get unique addresses from input and output and generate csv with list
unique_address_in = df_input['input_address'].drop_duplicates()
print(len(unique_address_in))
unique_address_out = df_output['address'].drop_duplicates()
print(len(unique_address_out))

all_address = unique_address_in.append(unique_address_out, ignore_index=True)
all_address = all_address.drop_duplicates().reset_index()

all_address = pd.DataFrame(all_address.iloc[:,1])
all_address.columns = ['address']
all_address.to_csv('.\\data\\unique_address.csv', header=None)

2905358
3310346


In [114]:
all_address

,address
0,13Uf71d8y94xEk2LX7GCtaBJmPiahhA7TR
1,166zajP74bcRVo7BmdeDME3mRX3Mi9e3xn
2,1LU3DtRE3XK32WxFqrnaT9k99nRgwHtLHd
3,1HVQNFf7vDpJVZk7tEzbFxnmALSezA2qPD
4,18vD11QAzCypY8RRx99ofY7obw5CSgnPWx
...,...
4709729,1LomBz6An18VzL5LqdrfENjiHSpSP6MxLP
4709730,1J2ak6wryyTxxfK7t5nbjedXQpq8CbBqod
4709731,1BF3dTey9tHbBBUJy7Kkx2sYWJXF4k3fvR
4709732,1HgytgHAo4ggrtyEbcZCm9spgJDLxfvMh3


### Input/Output high risk

In [115]:
input_labels = pd.DataFrame(pd.read_pickle('.\\data\\input_labels.pkl' ))
input_labels.columns = ['address']
output_labels = pd.DataFrame(pd.read_pickle('.\\data\\output_labels.pkl'))
output_labels.columns = ['address']

high_risk = pd.read_csv('.\\data\\high_risk.csv')

### Get extended data for labeled addresses

In [116]:
labels_data = pd.read_csv('.\\data\labels_extended_original.csv', 
                          chunksize = 1000000,
                          names=['address','id', 'entity','category','user_score'])

for chunk in labels_data:
    chunk = chunk[chunk['address'].isin(all_address['address'])]
    chunk.to_csv('.\\data\\labels_address.csv', mode='a', index=False, header=None)

### Check consistency

In [117]:
address = '1CAcQiErPxK4JixV6n77z4CkVagBp32E6q'
all_address[all_address.address == address]

,address
1174579,1CAcQiErPxK4JixV6n77z4CkVagBp32E6q


In [118]:
labels_extended[labels_extended.address == address]

,address,index,name,category,user_score
639,1CAcQiErPxK4JixV6n77z4CkVagBp32E6q,50,Exmo,Exchanges,1.00000
